<h2>C - Network

In [4]:
import pandas as pd
import icecream as ic
import numpy as np
import plotly
import matplotlib as plt
import xml.etree.ElementTree as ET
import pandas as pd
import psycopg2
import psycopg2.extras

C1: small-molecule roles

In [61]:
#chebi database

hostname = 'localhost'
database = 'Chebi'
username = 'postgres'
pwd = 'wlf1115'
port_id = 5432
conn = None

query = 'Acetazolamide'
with psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id) as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:

        query_script = '''SELECT type, final_id
                        FROM relation
                        WHERE init_id = (
                            SELECT compound_id
                            FROM names
                            WHERE name IN (%s)
                            )'''
        cur.execute(query_script, (query,))
        
        type_id = {}
        type_compound = {}
        for record in cur.fetchall():
            type_id[record[0]] = []
            type_compound[record[0]] = []
            type_id[record[0]] = record[1]


        for k,v in type_id.items():
            # query1 = '''SELECT name
            #             FROM names
            #             WHERE compound_id IN ({}
            #             )'''.format(tuple([x]))
            query_script1 = '''SELECT name
                        FROM names
                        WHERE compound_id IN (%s)
                        '''
            query1 = tuple([v])
            cur.execute(query_script1, query1)
            for record in cur.fetchall():
                type_compound[k].append(record)

print(query)
type_compound    

Acetazolamide


{'has_part': [['Diamox'],
  ['Sodium acetazolamide'],
  ['sodium [(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['N-(5-Sulfamoyl-1,3,4-thiadiazol-2-yl)acetamide monosodium salt']],
 'is_conjugate_base_of': [['[(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['acetazolamide']]}

C2: metabolic pathways

In [ ]:
# https://www.ebi.ac.uk/biomodels/MODEL1109130000#Files #!how to get binary data, how is the graph represented # read paper
# We downloaded the reconstruction of human metabolism (Recon)57 from Pathway Commons58 (http://www.pathwaycommons.org, July 2017) in binary interaction form. 
# Data were represented as an undirected graph where nodes are metabolites and edges denote reactions. 

In [120]:
recon_df = pd.read_csv("../PathwayCommons.8.reconx.BINARY_SIF.hgnc.txt", sep='\t', names=['init', 'process','affected'])
recon_df #how to convert protein name to abbreviation

,init,process,affected
0,A4GALT,catalysis-precedes,B3GALNT1
1,A4GALT,catalysis-precedes,B3GNT3
2,A4GALT,controls-production-of,CHEBI:58223
3,A4GALT,catalysis-precedes,ENTPD4
4,A4GALT,catalysis-precedes,NME1
...,...,...,...
52598,ZADH2,controls-production-of,CHEBI:17790
52599,ZADH2,controls-production-of,CHEBI:18041
52600,ZADH2,controls-production-of,CHEBI:28972
52601,ZADH2,catalysis-precedes,GLO1


C3: signaling pathways

In [ ]:
#  C3 space (and the C4 and C5 spaces) is aimed at any molecule with known protein targets. 
# In this case, we list the biological pathways that may be affected by the interaction of a molecule with its targets. 
# Human pathways were collected from Reactome (https://reactome.org, May 2017) #! use of graphDB

# we chose to use binding activities from B4, since this is an extensive dataset containing mostly literature data with well-accepted activity thresholds. 
# In B4, 24.5% of the compound–protein interactions do not correspond to human proteins. #!need B4 data
# These were mapped to their human orthologs using MetaPhOrs60 (http://orthology.phylomedb.org, May 2017), following the observation that binding activities can be safely transferred between orthologous proteins61,  #!how is it done, it is a database?
# especially if they belong to closely related species, as it is the case for B4 data62. Of all the nonhuman proteins mapped to the human orthologs, 94.4% were mammal proteins #!http://orthology.phylomedb.org/download

In [7]:
reactome_df = pd.read_csv("../UniProt2Reactome_All_Levels.txt", sep='\t', names=['uniprot_id','interaction_code','web','pathway','IEA','organism'])
reactome_df = reactome_df[['uniprot_id','pathway']]

chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
chemble_B1_target_uniprot = chemble_B1[chemble_B1['compound_name']=='Acetazolamide']['uniprot_id']



/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_1245/1950680321.py:4: DtypeWarning: Columns (5,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)


In [18]:
#  biological pathways that may be affected by the interaction of a molecule with its targets. 
# ! still need to make sure all the non human genes are turned into human orthologs
reactome_df[reactome_df['uniprot_id'].isin(chemble_B1_target_uniprot)]


,uniprot_id,pathway
412824,O00255,Signal Transduction
412825,O00255,Signaling by TGF-beta Receptor Complex
412826,O00255,Signaling by Rho GTPases
412827,O00255,RHO GTPase Effectors
412828,O00255,Signaling by WNT
...,...,...
903808,Q9Y6L6,Disorders of transmembrane transporters
903809,Q9Y6L6,Transport of organic anions
903810,Q9Y6L6,Metabolism of steroids
903811,Q9Y6L6,Drug ADME


C4: biological processes

In [6]:
with open('../GOA/25.H_sapiens.goa', 'r') as handle:

    for rec in gafiterator(handle):
        print(rec)
        break

{'DB': 'UniProtKB', 'DB_Object_ID': 'A0A024QZ33', 'DB_Object_Symbol': 'NSRP1', 'Qualifier': ['involved_in'], 'GO_ID': 'GO:0000381', 'DB:Reference': ['GO_REF:0000002'], 'Evidence': 'IEA', 'With': ['InterPro:IPR018612'], 'Aspect': 'P', 'DB_Object_Name': 'Nuclear speckle splicing regulatory protein 1', 'Synonym': ['NSRP1', 'CCDC55', 'hCG_1646942'], 'DB_Object_Type': 'protein', 'Taxon_ID': ['9606'], 'Date': '20220907', 'Assigned_By': 'InterPro', 'Annotation_Extension': '', 'Gene_Product_Form_ID': ''}


In [17]:
# We downloaded the Gene Ontology Annotation database (https://www.ebi.ac.uk/GOA, May 2017)
# read the ‘biological process’ branch of the ontology as a directed acyclic graph (‘is a’ relationships). 
# Proteins were annotated with their Gene Ontology Annotation biological process terms plus parent terms (up to the root of the directed acyclic graph). 
# Similar to C3, we associated molecules with biological process terms by simply checking the annotations of the molecule targets (B4).
from ast import And
from Bio.UniProt.GOA import gafiterator, record_has

for x in chemble_B1_target_uniprot.to_list():
    Interpro_ID = {'With': set([f'InterPro:IPR018612'])} 
    DB_Object_ID = {'DB_Object_ID': set([x])} 

    # DB_Object_name = {'DB_Object_name' : set(['Nuclear speckle splicing regulatory protein 1'])} 

    with open('../GOA/25.H_sapiens.goa', 'r') as handle:

        for rec in gafiterator(handle):
            if record_has(rec, DB_Object_ID):
                for key in ('DB_Object_ID','DB_Object_Name', 'Qualifier', 'GO_ID'):
                    print(rec[key])
                print('='*20)
        break

P00915
Carbonic anhydrase 1
['enables']
GO:0004064
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0005515
P00915
Carbonic anhydrase 1
['enables']
GO:0005515
P00915
Carbonic anhydrase 1
['enables']
GO:0008270
P00915
Carbonic anhydrase 1
['enables']
GO:0016829
P00915
Carbonic anhydrase 1
['enables']
GO:0016836
P00915
Carbonic anhydrase 1
['enables']
GO:0016836
P00915
Carbonic anhydrase 1
['enables']
GO:0018820
P00915
Carbonic anhydrase 1
['enables']
GO:0046872
P00915
Carbonic anhydrase 1
['involved_in']
GO:0006730
P00915
Carbonic anhydrase 1
['involved_in']
GO:0006730
P00915
Carbonic anhydrase 1
['is_active_in']
GO:0005737
P00915
Carbonic anhydrase 1
['located_in']
GO:0005737
P00915
Carbonic

C5: interactomes

In [68]:
# We collected five representative PPI networks, 
# namely STRING (score of >700, that is, high confidence)63 (v.10, https://string-db.org) (14,725 proteins (p), 300,686 interactions (i)), #! waiting for data
# InWeb (score of 0.5)64 (http://www.intomics.com/inbio/map, March 2017) (10,100 p, 168,970 i), #!paid service? <- not gonna use
# a portion of Pathway Commons containing interactions from known pathways (Kyoto Encylopedia of Genes and Genomes65, NetPath66, PANTHER67 and WikiPathways68) (9,344 p, 242,962 i), #!need to find out
# an in-house network of physical binary PPIs69 (13,038 p, 64,659 i),  #!need to find in house pipeline

In [69]:
stringDB_df = pd.read_csv('../human_HighConfidence_string_PPI.csv', sep=' ')

In [70]:
stringDB_df = stringDB_df[stringDB_df['combined_score']>700]

In [64]:
stringDB_df

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000324287,0,0,0,49,147,0,736,767
1,9606.ENSP00000000233,9606.ENSP00000387286,0,0,0,79,248,600,141,730
2,9606.ENSP00000000233,9606.ENSP00000262812,0,0,0,190,163,600,173,745
3,9606.ENSP00000000233,9606.ENSP00000158762,0,0,0,0,147,0,736,765
4,9606.ENSP00000000233,9606.ENSP00000449270,0,0,0,162,210,600,194,757
...,...,...,...,...,...,...,...,...,...,...
504021,9606.ENSP00000485663,9606.ENSP00000248342,0,0,0,699,996,900,957,999
504022,9606.ENSP00000485663,9606.ENSP00000416255,0,0,0,167,808,0,473,908
504023,9606.ENSP00000485663,9606.ENSP00000220849,0,0,0,907,993,900,896,999
504024,9606.ENSP00000485663,9606.ENSP00000346088,0,0,0,711,0,0,131,738


In [60]:
stringDB_df.columns

Index(['protein1 protein2 neighborhood fusion cooccurence coexpression experimental database textmining combined_score'], dtype='object')

In [67]:
# an in-house network of physical binary PPIs69 (13,038 p, 64,659 i),  #!need to find in house pipeline
import csv
# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("../interactions_interactome_3d.dat").readlines()]
interaction3D_df = pd.DataFrame(datContent, columns = datContent[0])
interaction3D_df = interaction3D_df[1:] 
interaction3D_df #! how did you get binary PPI interactome info from this?  https://interactome3d.irbbarcelona.org/help.php#interaction_page


,PROT1,PROT2,RANK_MAJOR,RANK_MINOR,TYPE,PDB_ID,BIO_UNIT,CHAIN1,MODEL1,SEQ_IDENT1,...,SEQ_END1,DOMAIN1,CHAIN2,MODEL2,SEQ_IDENT2,COVERAGE2,SEQ_BEGIN2,SEQ_END2,DOMAIN2,FILENAME
1,A0A024RAV5,P02647,1,0,Structure,2mse,1,B,0,100.0,...,185,-,A,0,100.0,74.2,68,265,-,A0A024RAV5-P02647-EXP-2mse.pdb1-B-0-A-0.pdb
2,A0A024RAV5,P02647,2,0,Structure,6pts,1,B,0,100.0,...,185,-,A,0,100.0,74.2,68,265,-,A0A024RAV5-P02647-EXP-6pts.pdb1-B-0-A-0.pdb
3,A0A024RAV5,P10398,1,0,Structure,2mse,1,B,0,100.0,...,185,-,D,0,100.0,12.0,19,91,-,A0A024RAV5-P10398-EXP-2mse.pdb1-B-0-D-0.pdb
4,A0A075B5G3,Q8NBP7,1,0,Structure,4ov6,1,F,0,100.0,...,99,-,B,0,95.6,42.5,153,446,-,A0A075B5G3-Q8NBP7-EXP-4ov6.pdb1-F-0-B-0.pdb
5,A0A075B5G3,Q8NBP7,2,0,Structure,4ov6,2,G,0,100.0,...,96,-,E,0,92.2,42.3,153,445,-,A0A075B5G3-Q8NBP7-EXP-4ov6.pdb2-G-0-E-0.pdb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124919,V9HW83,V9HW83,54,0,Structure,4wb9,1,A,2,100.0,...,501,-,A,3,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-2-A-3.pdb
124920,V9HW83,V9HW83,55,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,1,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-1.pdb
124921,V9HW83,V9HW83,56,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,2,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-2.pdb
124922,V9HW83,V9HW83,57,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,3,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-3.pdb


In [ ]:
# a network of metabolic genes based on Recon (v.2, http://vmh.uni.lu) (1,628 p, 246,937 i). #! recon https://www.vmh.life/#home matlab file?
import scipy.io
Recon3DModel_mat = scipy.io.loadmat('../Recon3D_301/Recon3DModel_301.mat')
Recon3D_mat = scipy.io.loadmat('../Recon3D_301/Recon3D_301.mat')



# To build this last network, we linked two metabolic proteins (enzymes or transporters) when the product metabolite of the first was the substrate of the second, 
# or when both were needed to perform a certain reaction, suggesting that they are part of the same protein complex. 
# Edges between proteins were weighted inversely proportional to the number of reactions involving their shared metabolites, 
# so that ‘currency’ metabolites such as ATP and water had marginal impact on the network connectivity. To control for indirect associations, 
# we deconvoluted the network using edge weights and setting a network deconvolution score cutoff of 0.9

In [24]:
Recon3D_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Recon3D'])

In [90]:
len(Recon3D_mat['Recon3D'][0][0][3])


8399

In [ ]:
Recon3D_mat['Recon3D'][0][0][1].tolist()

In [100]:
with open('recon_3d.txt', 'w') as f:
    writer = csv.writer(f)
    for x in range(1,len(Recon3D_mat['Recon3D'][0][0])):
        writer.writerow(Recon3D_mat['Recon3D'][0][0][x].tolist())


In [ ]:
Recon3D_mat['Recon3D'][0][0][6].tolist()

In [94]:
Recon3D_mat['Recon3D'][0][0][4]

array([[array(['10FTHF5GLUtl'], dtype='<U12')],
       [array(['10FTHF5GLUtm'], dtype='<U12')],
       [array(['10FTHF6GLUtl'], dtype='<U12')],
       ...,
       [array(['NADH2_u10mi'], dtype='<U11')],
       [array(['CYOOm3i'], dtype='<U7')],
       [array(['CYOOm2i'], dtype='<U7')]], dtype=object)

In [93]:
len(Recon3D_mat['Recon3D'][0][0][1])

8399

In [ ]:
Recon3DModel_mat

In [65]:
#network processing


Hot net